In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os, time, shutil, sys
from pathlib import Path
from glob import glob
from pprint import pprint
import numpy as np
import spikeinterface.full as si
import matplotlib.pyplot as plt

sys.path.append("/home/phornauer/Git/axon_tracking/")
from axon_tracking import spike_sorting as ss

In [ ]:
sorter = 'kilosort2_5'
si.Kilosort2_5Sorter.set_kilosort2_5_path('/home/phornauer/Git/Kilosort_2020b') #Change
sorter_params = si.get_default_sorter_params(si.Kilosort2_5Sorter)
sorter_params['n_jobs'] = -1
sorter_params['detect_threshold'] = 7
sorter_params['minFR'] = 0.01
sorter_params['minfr_goodchannels'] = 0.01
sorter_params['keep_good_only'] = False
sorter_params['do_correction'] = False
#pprint(sorter_params)

In [ ]:
rec_path = '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/phornauer/231207/Chemogenetics_2_Row_2/T002443/AxonTracking/000014/data.raw.h5'
stream_id = 'well000'

In [ ]:
parent_path = '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/manuelsc/'
recording_date = '231207'
project_name = 'D*_*1'
chip_id = '*'

assay_name = 'Network'
assay_id = '0*'
file_name = 'data.raw.h5'
path_parts = [parent_path,  project_name, recording_date, chip_id, assay_name, assay_id, file_name]
path_list, stream_ids = ss.generate_rec_list(path_parts)
pprint(path_list)

In [ ]:
path_list = [path_list[x] for x in [0]]

In [ ]:
axon_rec, ax_el = ss.concatenate_recording_slices(rec_path, stream_id)

In [ ]:
network_rec = ss.concatenate_recording_list(path_list, stream_id)

In [ ]:
nw_rec_el = network_rec.get_property("contact_vector")["electrode"]
ax_rec_el = axon_rec.get_property("contact_vector")["electrode"]

common_el = list(set(ax_rec_el).intersection(nw_rec_el))

chan_idx = [np.where(ax_rec_el == el)[0][0] for el in common_el]
ax_channels = axon_rec.get_channel_ids()[chan_idx]
ax_rec_slice = axon_rec.channel_slice(ax_channels, renamed_channel_ids=list(range(len(chan_idx))))

rec_list = [ax_rec_slice.astype('float16')]

chan_idx = [np.where(nw_rec_el == el)[0][0] for el in common_el]
nw_channels = network_rec.get_channel_ids()[chan_idx]
nw_rec_slice = network_rec.channel_slice(nw_channels, renamed_channel_ids=list(range(len(chan_idx))))

rec_list.append(nw_rec_slice.astype('float16'))

In [ ]:
nw_double_slice = network_rec.frame_slice(start_frame=1800*10000,end_frame=nw_rec_slice.get_num_samples())

In [ ]:
test_nw = network_rec.channel_slice(nw_channels, renamed_channel_ids=list(range(len(chan_idx))))

In [ ]:
test_nw

In [ ]:
test_nw.save_to_folder(folder='/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_2/concatenated/test',n_jobs=-1)

In [ ]:
concatenated_recording = si.concatenate_recordings(rec_list)
concatenated_recording

In [ ]:
save_root = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_2/concatenated/'
ss.clean_sorting(rec=concatenated_recording, save_root=save_root, stream_id=stream_id, sorter=sorter, sorter_params=sorter_params)

In [ ]:
#'/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/Chemogenetics/231205/2977/Cell_B_positive.raw.h5'
rec_path, stream_id, rec_name = path_list[0],'well000','rec0000'
rec = si.MaxwellRecordingExtractor(rec_path, stream_id=stream_id, rec_name=rec_name)

In [ ]:
rec

In [ ]:
sel_channels = np.random.choice(rec.get_channel_ids(),size=100,replace=False)

In [ ]:
slice_rec = rec.channel_slice(sel_channels, renamed_channel_ids=list(range(len(sel_channels))))
slice_rec

In [ ]:
save_path = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_2/concatenated/test3'
slice_rec.save_to_folder(folder=save_path, n_jobs=-1)